In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import numpy as np

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

from itertools import chain

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [2]:
consumer_key= 'QfCbmtE2LHx78PltOQOtd85u0'
consumer_secret= 'Cn8IcNEpXctL3B7WllOFk8LE8lCa8VzRGyZzRwKlJRRPFGcw3P'
access_token= '1460337339410964489-ktZ3iJQ8CY5Rk5x37Lj8mKlD6kpZJF'
access_token_secret= '9k6appXQpDdknghnkM9mq9PMitDwbv0CEGj4jP4sNv0cS'
bearer_token= 'AAAAAAAAAAAAAAAAAAAAAJV1VwEAAAAAoHauZsujJtDsb0Q9Z74S%2B9bUph0%3D3Afav22UHMJDnr98dvWisXhLHFqeohCt1b81H63IWm54AOxkSY'

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
client = tw.Client(bearer_token=bearer_token)

query = 'from:news'

tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'], max_results=100)

for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

In [5]:
# Replace with your own search query
query = 'news'

# Replace the limit=1000 with the maximum number of Tweets you want
tweets= tw.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1000)

all_tweets = [tweet.text for tweet in tweets]



Rate limit exceeded. Sleeping for 615 seconds.


In [ ]:
query = 'covid'

counts = client.get_recent_tweets_count(query=query, granularity='day')

tweetCount = 0

for count in counts.data:
    tweetCount = tweetCount + count['tweet_count']

print('Covid tweet count in the last 7 days - ' + str(tweetCount))

In [ ]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
all_tweets_no_urls = [remove_url(tweet) for tweet in all_tweets]

In [ ]:
words_in_tweet = [tweet.lower().split() for tweet in all_tweets_no_urls]

In [ ]:
all_words_no_urls = list(chain(*words_in_tweet))
counts_no_urls = collections.Counter(all_words_no_urls)
counts_no_urls.most_common(15)

In [ ]:
clean_tweets_no_urls = pd.DataFrame(counts_no_urls.most_common(15),
                             columns=['words', 'count'])

clean_tweets_no_urls.head()

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))

stop_words.add('rt')

# View a few words from the set
list(stop_words)[0:10]

In [ ]:
tweets_nsw = [[word for word in tweet_words if not word in stop_words]
              for tweet_words in words_in_tweet]

tweets_nsw[0]

In [ ]:
all_words_nsw = list(chain(*tweets_nsw))

counts_nsw = collections.Counter(all_words_nsw)

counts_nsw.most_common(15)

In [ ]:
clean_tweets_nsw = pd.DataFrame(counts_nsw.most_common(15),
                             columns=['words', 'count'])
clean_tweets_nsw.head()

In [ ]:
tweet_hashtags = [re.findall(r"#(\w+)", tweet.lower()) for tweet in all_tweets]
all_hashtags = []
for hashtags in tweet_hashtags:
    all_hashtags.extend(hashtags)

all_hashtags

In [ ]:
counts_hashtags = collections.Counter(all_hashtags)
counts_hashtags.most_common(15)

In [ ]:
hashtag_counts = pd.DataFrame(counts_hashtags.most_common(15),
                             columns=['hashtags', 'count'])
hashtag_counts.head()

In [3]:
client = tw.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# Replace with your own search query
query = 'news'

# Replace the limit=1000 with the maximum number of Tweets you want
tweets= tw.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1000)

tweet_ids = [{'id': tweet.id,'text': tweet.text} for tweet in tweets]
tweet_ids

[{'id': 1466539397340622848,
  'text': 'RT @nipponkairagi: 選挙戦の大きな争点として、ジェンダー平等を掲げて訴えた。立憲民主党の蓮舫議員も駆けつけ「＃女性の声が政治を変える」という街頭演説会も開いた。 だが選挙戦が終わった時、男性たちから「女、女と言っているから落ちたんだ」と言われた。\n\nその通りで…'},
 {'id': 1466539397176643585,
  'text': 'RT @RightWingCope: He literally shot up a school and this news site went with an old picture of him praying.\n\nAmerican media is insane. htt…'},
 {'id': 1466539396715323400,
  'text': 'RT @ShoebridgeMLC: If you stand for nothing then stand aside Dave Sharma. This is classic ‘narrow casting’ where a politician gives differe…'},
 {'id': 1466539396690489344,
  'text': 'RT @entameclipnews: 電話を通じて”素”を切り取る「RING³」(リン・リン・リン)第7回に私立恵比寿中学から安本彩花が初登場！ 電話相手はももいろクローバーZのリーダー百田夏菜子！\n\n記事はこちら(画像4枚)🖇\nhttps://t.co/FV9bYFHQlc…'},
 {'id': 1466539396685983751,
  'text': "RT @itvnews: In ITV News' eight-month investigation, tenants have reported racism in their dealings with councils over problems with their…"},
 {'id': 1466539396581130243,
  'text': 'RT @RWMaloneMD: Yet another exampl

In [4]:
tweet_counts = []
for tweet in tweet_ids:
    users = tw.Paginator(client.get_retweeters, id=tweet['id']).flatten(limit=1000)
    usernames = [user.name for user in users]
    print(len(usernames))

Unexpected parameter: pagination_token
Rate limit exceeded. Sleeping for 697 seconds.
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


0
0
0


Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


0
0
0


Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


...

0
0


Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


0
0


Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


0
0
0


In [ ]:
for tweet in tweet_ids:
    users = client.get_retweeters(id=id)
    if not users:
        for user in users.data:
            print(user)